<b>Partie IA</b>

Imports

In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd  # Pour manipuler les données sous forme de DataFrame
import warnings  # Pour gérer les avertissements

# Importation des modules de Scikit-learn pour la transformation des colonnes, pipelines, modèles et évaluation
from sklearn.compose import ColumnTransformer  # Pour appliquer des transformations différentes sur les colonnes
from sklearn.pipeline import Pipeline  # Pour créer des pipelines de transformations et de modèles
from sklearn.preprocessing import StandardScaler  # Pour le prétraitement des données
from sklearn.model_selection import train_test_split  # Pour la division des données et la recherche des hyperparamètres
from sklearn.metrics import accuracy_score, classification_report  # Pour évaluer les performances du modèle
from catboost import CatBoostClassifier  # Classifieur CatBoost

# Ignorer certains avertissements
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import sklearn

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
df_tr = pd.read_csv(r'C:\Users\colin\Documents\ESEO\E5\docsPFE\creditcard_2023.csv')

L'étude des variables sur la partie Big Data nous a permis de voir que certaines corrélations de variables sont plus importantes que d'autres dans le dataset pour les transactions frauduleuse. Car nous ne pouvont pas être 100% sûrs de pouvoir enelever celles qui n'ont pas l'air importantes, nous les gardons.

On teste plusieurs méthodes d'intelligence artificielle pour en trouver une qui est à la fois rapide et efficace.

*Apprentissage supervisé*

In [3]:
# Séparation des données en caractéristiques (X) et cible (y)
X = df_tr.drop('Class', axis=1)
y = df_tr['Class']

In [4]:
# Définir les caractéristiques numériques (exclure les colonnes catégorielles)
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [5]:
# Définir le pipeline de transformation pour les colonnes numériques
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

StandardScaler() est un transformateur qui normalise les données en les centrant sur la moyenne 0 et en les réduisant à une échelle unitaire (écart-type 1). Cela est souvent nécessaire avant de passer à des algorithmes d'apprentissage automatique, car certains modèles sont sensibles à l'échelle des données (par exemple, les modèles basés sur la distance comme la régression logistique, SVM, etc.).

In [6]:
# Définir la transformation par colonnes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

In [7]:
# Définir le modèle
model = Pipeline(steps=[('preprocessor', preprocessor),
                         ('classifier', CatBoostClassifier(verbose=False))])

In [8]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
# Entraîner le modèle
model.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['id', 'V1', 'V2', 'V3', 'V4',
                                                   'V5', 'V6', 'V7', 'V8', 'V9',
                                                   'V10', 'V11', 'V12', 'V13',
                                                   'V14', 'V15', 'V16', 'V17',
                                                   'V18', 'V19', 'V20', 'V21',
                                                   'V22', 'V23', 'V24', 'V25',
                                                   'V26', 'V27', 'V28',
                                                   'Amount'])])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x000002134C35E2C0>)])

In [10]:
# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)

In [11]:
# Calculer l'exactitude
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9996570705027874


In [12]:
# Afficher le rapport de classification avec couleurs et en-tête
from colorama import Fore, Back, Style
print("\n" + Back.BLUE + Fore.WHITE + "Classification Report" + Style.RESET_ALL)
report = classification_report(y_test, y_pred, output_dict=True)
for key, value in report.items():
    if key in ['0', '1']:  
        color = Fore.GREEN if value['precision'] > 0.8 else Fore.RED
        print(f"Class {key}:")
        print(f"  Precision: {color}{value['precision']:.2f}{Style.RESET_ALL}")
        color = Fore.GREEN if value['recall'] > 0.8 else Fore.RED
        print(f"  Recall: {color}{value['recall']:.2f}{Style.RESET_ALL}")
        color = Fore.GREEN if value['f1-score'] > 0.8 else Fore.RED
        print(f"  F1-score: {color}{value['f1-score']:.2f}{Style.RESET_ALL}")
        print(f"  Support: {value['support']}")
    else:
        print(key + ":", value)


Classification Report
Class 0:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
  Support: 56750.0
Class 1:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
  Support: 56976.0
accuracy: 0.9996570705027874
macro avg: {'precision': 0.9996569545708367, 'recall': 0.9996571914660413, 'f1-score': 0.9996570692311153, 'support': 113726.0}
weighted avg: {'precision': 0.999657078158671, 'recall': 0.9996570705027874, 'f1-score': 0.9996570705434346, 'support': 113726.0}


*Forêt aléatoire*

On va donc séparer nos données en un échantillon d’apprentissage pour apprendre notre modèle et un échantillon test pour tester l’efficacité de la forêt aléatoire.

In [13]:
x_train, x_test, y_train, y_test  = train_test_split(X, 
                                                     y, 
                                                     test_size=0.25, 
                                                     random_state=42)

On utilise un échantillon de test représentant 25% des données.

On va utiliser scikit-learn pour construire notre forêt aléatoire, on utilise donc la classe RandomForestClassifier de scikit-learn et on crée un objet à partir de cette classe. C’est à ce moment que nous allons définir ce qu’on appelle les hyperparamètres du modèle. Il s’agit de paramètres tels que le nombre d’arbres dans la forêt qui doivent être définis en amont de l’apprentissage.

In [14]:
modele_rf = RandomForestClassifier(
     n_estimators=100,
     criterion='gini',
     max_depth=None,
     min_samples_split=2,
     min_samples_leaf=1,
     min_weight_fraction_leaf=0.0,
     max_features='sqrt',
     max_leaf_nodes=None,
     min_impurity_decrease=0.0,
     bootstrap=True,
     oob_score=False,
     n_jobs=None,
     random_state=None,
     verbose=0,
     warm_start=False,
     class_weight=None,
     ccp_alpha=0.0,
     max_samples=None,)

Tous ces hyperparamètres ont des valeurs par défaut que nous modifions rarement, mais il est utile de les comprendre. En voici un description rapide :

n_estimators : il s’agit du nombre d’arbres dans la forêt
criterion : il s’agit du critère utilisé pour construire les arbres et séparer les branches des arbres

max_depth : il s’agit de la profondeur maximale des arbres utilisés (le nombre de niveaux dans l’arbre de décision)

min_sample_split : il s’agit du nombre d’échantillons minimal dans une feuille pour refaire une séparation

min_samples_leaf : il s’agit du nombre d’échantillons minimal pour créer une feuille

min_weight_fraction_leaf : il s’agit de la fraction du nombre total d’échantillon minimal pour créer une feuille

max_features : il s’agit du nombre de colonnes sélectionnées pour chaque arbre (par défaut on prend la racine carré du nombre de colonnes)

max_leaf_nodes : il s’agit du nombre maximal de feuilles

min_impurity_decrease : il s’agit de la baisse minimale du critère d’impureté pour faire une séparation

bootstrap : paramètre pour utiliser du bootstrap, si il est à False, le même échantillon est pris pour chaque arbre

n_jobs ; nombre de traitements à effectuer en parallèle
random_state : graine aléatoire

warm_start : ceci permet de repartir du résultat du dernier apprentissage pour faire l’apprentissage

class_weights : il s’agit des poids associés à chaque classe si cela a un sens

max_samples : si vous voulez réduire le nombre d’observations dans vos échantillons bootstrap


Nous avons donc construit un objet forêt aléatoire, nous devons maintenant effectuer l’apprentissage :

In [15]:
modele_rf.fit(x_train, y_train)

RandomForestClassifier()

On peut afficher l’importance des variables de notre modèle de forêt aléatoire :

In [16]:
pd.DataFrame(modele_rf.feature_importances_,
              index = x_train.columns, 
              columns = ["importance"]).sort_values(
     "importance", 
     ascending = False)

,importance
id,0.348717
V14,0.126415
V4,0.114920
V10,0.090378
V12,0.081691
V17,0.063336
V3,0.039081
V16,0.035598
V11,0.031341
V21,0.013466


Une fois notre modèle appris, il faut le valider. Pour cela, on va utiliser les données de test et des indicateurs très classiques.

On compare donc la valeur prédite par le modèle avec la valeur dans les données. Il s’agit ici de comparer la couleur du vin prédite par le modèle et celle observée dans notre échantillon test.

Le premier indicateur utilisé est le pourcentage de bien classé aussi appelé accuracy :

In [17]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(f"Le pourcentage de bien classés est de : {accuracy_score(y_test, modele_rf.predict(x_test))*100} %")

Le pourcentage de bien classés est de : 99.98241393379197 %


https://www.stat4decision.com/fr/foret-aleatoire-avec-python/

*Réseau de neuronnes*

In [49]:
# Convert pandas DataFrame and Series to PyTorch tensors
X_nn = torch.tensor(X.values, dtype=torch.float32)  # Features as tensor
y_nn = torch.tensor(y.values, dtype=torch.float32).reshape(-1, 1)  # Target as tensor

In [60]:
# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [61]:
# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_test = torch.tensor(scaler.transform(X_test), dtype=torch.float32)

# Convert y_train and y_test to tensors
y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float32).reshape(-1, 1)  # Ensure correct shape for binary classification
y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32).reshape(-1, 1)


In [62]:
# Define the model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.hidden1 = nn.Linear(30, 12)  # Input layer (30 features) to first hidden layer (12 neurons)
        self.act1 = nn.ReLU()  # Activation function
        self.hidden2 = nn.Linear(12, 8)  # Second hidden layer (8 neurons)
        self.act2 = nn.ReLU()  # Activation function
        self.output = nn.Linear(8, 1)  # Output layer (1 neuron for binary classification)
        self.act_output = nn.Sigmoid()  # Sigmoid activation for binary output

    def forward(self, x):
        x = self.hidden1(x)
        x = self.act1(x)
        x = self.hidden2(x)
        x = self.act2(x)
        x = self.output(x)
        x = self.act_output(x)
        return x


In [63]:
# Initialize model, loss function, and optimizer
model = SimpleNN()
loss_fn = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer


In [64]:
# Training the model
num_epochs = 100

In [65]:
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    
    # Backward pass and optimization
    optimizer.zero_grad()  # Clear gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights
    
    if (epoch + 1) % 10 == 0:  # Print every 10 epochs
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')


Epoch 10/100, Loss: 0.6810497641563416
Epoch 20/100, Loss: 0.6551823019981384
Epoch 30/100, Loss: 0.6253800988197327
Epoch 40/100, Loss: 0.5925559997558594
Epoch 50/100, Loss: 0.5592091083526611
Epoch 60/100, Loss: 0.5280115008354187
Epoch 70/100, Loss: 0.5005115866661072
Epoch 80/100, Loss: 0.476925253868103
Epoch 90/100, Loss: 0.45691806077957153
Epoch 100/100, Loss: 0.4396722614765167


In [66]:
# Evaluation
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test)
    y_test_pred = (y_test_pred > 0.5).float()  # Convert probabilities to binary output


In [67]:
# Calculate accuracy
accuracy = accuracy_score(y_test.numpy(), y_test_pred.numpy())
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9359


In [68]:
# Classification report
print("Classification Report:")
print(classification_report(y_test.numpy(), y_test_pred.numpy()))

Classification Report:
              precision    recall  f1-score   support

         0.0       0.92      0.96      0.94     56750
         1.0       0.96      0.91      0.93     56976

    accuracy                           0.94    113726
   macro avg       0.94      0.94      0.94    113726
weighted avg       0.94      0.94      0.94    113726



https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/

*Gradient Boost*

In [73]:
!pip install xgboost

  Using cached xgboost-2.1.2-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-2.1.2-py3-none-win_amd64.whl (124.9 MB)


In [74]:
# multiclass classification
import pandas
import xgboost
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [76]:
# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y)
label_encoded_y = label_encoder.transform(y)
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, label_encoded_y, test_size=test_size, random_state=seed)

In [77]:
# fit model no training data
model = xgboost.XGBClassifier()
model.fit(X_train, y_train)
print(model)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)


In [78]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.98%


https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/